In [1]:
from transformers import RobertaTokenizer, RobertaModel
import torch

# Carga el tokenizador y el modelo
#tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
#model = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model_roberta = RobertaModel.from_pretrained('roberta-base')

# Oración de ejemplo
sentence = "Hola, esto es una prueba con BERT."

# Tokeniza la oración
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Obtiene los embeddings
with torch.no_grad():
    outputs = model_roberta(**inputs)

# Los embeddings de la última capa oculta serían
last_hidden_states = outputs.last_hidden_state

# Embedding del [CLS] token que representa la oración entera
sentence_embedding = last_hidden_states[:, 0, :]

# Embeddings de cada token/palabra en la oración
word_embeddings = last_hidden_states

print("Embedding de la oración:", sentence_embedding)
print("Embeddings de las palabras:", word_embeddings)


/home/spoturno/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding de la oración: tensor([[-8.6743e-02,  9.0471e-02,  1.3302e-02, -1.0595e-01,  3.9593e-02,
         -1.7688e-02, -4.2758e-02, -2.3704e-03,  3.3589e-02, -1.0090e-01,
         -4.4411e-02, -1.7645e-02,  7.2555e-02, -4.6160e-02,  7.3803e-02,
          2.9888e-02, -5.5485e-02, -2.1723e-02, -2.7182e-02, -3.6914e-02,
         -1.0021e-01,  1.2966e-02, -9.6302e-03,  9.4810e-02, -2.5938e-02,
          8.1167e-02,  8.1004e-02,  8.1191e-02, -6.7600e-02,  5.5690e-02,
         -5.8468e-02, -7.9548e-02,  5.6678e-02,  1.0321e-02,  6.8747e-02,
          1.3197e-01,  4.0268e-02, -2.0073e-02, -4.9358e-02,  9.8386e-03,
         -7.8971e-03,  1.7162e-01, -1.2055e-02,  1.9111e-02,  6.3142e-02,
         -3.0365e-03,  2.0137e-02, -6.0194e-03, -3.0871e-02,  2.8043e-02,
          4.6304e-02,  6.4634e-02, -3.9413e-02,  3.3905e-02, -1.8681e-01,
          4.6870e-02,  5.5762e-02,  7.0085e-02, -3.2284e-02, -8.9200e-02,
         -3.3546e-02, -1.7492e-01, -9.4362e-02, -8.4185e-02,  7.9824e-02,
         -6.4

In [2]:
tokenizer

RobertaTokenizer(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [3]:
last_hidden_states.shape

torch.Size([1, 17, 768])

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
import json
# Supongamos que tus datos están en un archivo JSON llamado 'converted_data.jsonl'
with open('../scripts/converted_data.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

# Convertir los datos en un DataFrame de pandas
df = pd.DataFrame(data)

In [5]:
#rename columns
df.rename(columns={'sentence': 'sentence', 'token': 'target_word', 'complexity': 'complexity'}, inplace=True)
df.head()

,sentence,target_word,complexity
0,"Behold, there came up out of the river seven c...",river,0.000000
1,I am a fellow bondservant with you and with yo...,brothers,0.000000
2,"The man, the lord of the land, said to us, 'By...",brothers,0.050000
3,Shimei had sixteen sons and six daughters; but...,brothers,0.150000
4,"""He has put my brothers far from me.",brothers,0.263889


In [6]:
import torch

device = torch.device("cpu")

model_roberta.to(device)


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Suponiendo que df es tu DataFrame que incluye las columnas 'sentence', 'target_word' y 'complexity'
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

from tqdm.auto import tqdm

def get_embeddings(text_list, tokenizer, model, device):
    """Obtiene embeddings [CLS] para una lista de textos, con barra de progreso."""
    model.eval()
    embeddings = []
    for text in tqdm(text_list, desc="Processing", leave=True):
        encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            output = model(**encoded_input)
        embeddings.append(output.last_hidden_state[:, 0, :].squeeze().cpu())
    return torch.stack(embeddings)


def process_dataframe(df, tokenizer, model, device):
    """Procesa un dataframe para obtener embeddings y complejidades."""
    sentence_embeddings = get_embeddings(df['sentence'].tolist(), tokenizer, model, device)
    word_embeddings = get_embeddings(df['target_word'].tolist(), tokenizer, model, device)
    complexities = torch.tensor(df['complexity'].values, dtype=torch.float).unsqueeze(1)
    return torch.cat((sentence_embeddings, word_embeddings, complexities), dim=1)

# Procesar los conjuntos de entrenamiento y prueba
train_data = process_dataframe(train_df, tokenizer, model_roberta, device)
test_data = process_dataframe(test_df, tokenizer, model_roberta, device)

Processing: 100%|██████████| 1533/1533 [00:44<00:00, 34.30it/s]


In [8]:
train_data.shape, test_data.shape

(torch.Size([6129, 1537]), torch.Size([1533, 1537]))

In [9]:
# Entradas de entrenamiento (todas las columnas excepto la última)
train_features = train_data[:, :-1]
# Objetivos de entrenamiento (última columna)
train_targets = train_data[:, -1]

# Entradas de prueba
test_features = test_data[:, :-1]
# Objetivos de prueba
test_targets = test_data[:, -1]

# Asegúrate de que los tensores estén en la CPU para convertirlos a arrays de NumPy
train_features_np = train_features.numpy()
train_targets_np = train_targets.numpy()
test_features_np = test_features.numpy()
test_targets_np = test_targets.numpy()


In [10]:
import tensorflow as tf

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

2024-03-23 13:29:21.173390: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 13:29:21.173797: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 13:29:21.176327: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 13:29:21.202044: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-23 13:29:21.828882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [11]:
import tensorflow as tf

# Definir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(train_features.shape[1],)),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=rmse,
              metrics=['mean_absolute_error'])

model.summary()

/home/spoturno/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 426,497 (1.63 MB)

 Trainable params: 426,497 (1.63 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Entrenamiento del modelo
history = model.fit(train_features_np, train_targets_np,
                    validation_split=0.2,
                    epochs=5,
                    batch_size=32)

Epoch 1/5
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1961 - mean_absolute_error: 0.1634 - val_loss: 0.1310 - val_mean_absolute_error: 0.0989
Epoch 2/5
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1233 - mean_absolute_error: 0.0964 - val_loss: 0.1141 - val_mean_absolute_error: 0.0867
Epoch 3/5
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1101 - mean_absolute_error: 0.0860 - val_loss: 0.1087 - val_mean_absolute_error: 0.0856
Epoch 4/5
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1115 - mean_absolute_error: 0.0868 - val_loss: 0.1081 - val_mean_absolute_error: 0.0831
Epoch 5/5
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1079 - mean_absolute_error: 0.0845 - val_loss: 0.1065 - val_mean_absolute_error: 0.0842


In [13]:
# Evaluación del modelo
test_loss, test_mae = model.evaluate(test_features_np, test_targets_np, verbose=2)
print(f"Test RMSE: {test_loss}, Test MAE: {test_mae}")

48/48 - 0s - 836us/step - loss: 0.1084 - mean_absolute_error: 0.0843
Test RMSE: 0.10837385058403015, Test MAE: 0.0842542052268982


In [14]:
model.save("modelos/roberta.keras")


In [16]:
import pandas as pd

input_file_path = '/home/spoturno/coding/MLSP_Data/Data/Test/English/multilex_test_en_lcp_unlabelled.tsv'
output_file_path = '/home/spoturno/coding/MLSP/predictions/roberta_with_mlp.tsv'

df = pd.read_csv(input_file_path, delimiter='\t', header=None, names=['en', 'english', 'sentence', 'target_word', 'complexity'])

df.head()

,en,english,sentence,target_word,complexity
0,en_31,english,After Ron nearly dies drinking poisoned mead t...,distraught,NaN
1,en_32,english,After Ron nearly dies drinking poisoned mead t...,drinking,NaN
2,en_33,english,"After the war, Hitler remained in the army and...",oratory,NaN
3,en_34,english,"After the war, Hitler remained in the army and...",reporting,NaN
4,en_35,english,"After the war, Hitler remained in the army and...",infiltrating,NaN


In [17]:
trial_set = process_dataframe(df, tokenizer, model_roberta, device)
trial_set = trial_set[:, :-1]

Processing: 100%|██████████| 570/570 [00:17<00:00, 33.38it/s]


In [18]:
prediction = model.predict(trial_set.numpy())
prediction

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[0.29537797],
       [0.29974836],
       [0.36869442],
       [0.2598371 ],
       [0.32287806],
       [0.29097232],
       [0.5070837 ],
       [0.32218733],
       [0.4086039 ],
       [0.32529747],
       [0.29130182],
       [0.3068468 ],
       [0.3155885 ],
       [0.32339245],
       [0.255053  ],
       [0.25225905],
       [0.2421126 ],
       [0.23818764],
       [0.23593727],
       [0.22156873],
       [0.48081815],
       [0.30930048],
       [0.2359544 ],
       [0.40036413],
       [0.3137411 ],
       [0.3955433 ],
       [0.3530424 ],
       [0.25066715],
       [0.24589984],
       [0.24194309],
       [0.3108501 ],
       [0.34966725],
       [0.27921954],
       [0.3507047 ],
       [0.39868677],
       [0.33516228],
       [0.28673303],
       [0.30933803],
       [0.2657011 ],
       [0.29323632],
       [0.23321757],
       [0.33427608],
       [0.29462826],
       [0.31695133],
       [0.41715893],
       [0.47356415],
       [0.47624847],
       [0.325

In [19]:

df['complexity'] = prediction

In [20]:
df.head()

,en,english,sentence,target_word,complexity
0,en_31,english,After Ron nearly dies drinking poisoned mead t...,distraught,0.295378
1,en_32,english,After Ron nearly dies drinking poisoned mead t...,drinking,0.299748
2,en_33,english,"After the war, Hitler remained in the army and...",oratory,0.368694
3,en_34,english,"After the war, Hitler remained in the army and...",reporting,0.259837
4,en_35,english,"After the war, Hitler remained in the army and...",infiltrating,0.322878


In [22]:
df.to_csv(output_file_path, index=False, sep='\t')